# L2: Create Agents to Research and Write an Article

Foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2 
)

2024-07-17 19:24:46,012 - 140531272817536 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan on Artificial Intelligence

Outline:
I. Introduction
- Definition of Artificial Intelligence (AI)
- Importance and applications of AI in various industries
- Preview of key trends and players in the AI field

II. Latest Trends in Artificial Intelligence
- Advancements in machine learning algorithms
- Increased adoption of AI in healthcare, finance, and retail sectors
- Ethical considerations in AI development
- Impact of AI on job market and workforce automation

III. Key Players in Artificial Int

I now can give a great answer

Final Answer: 

# The Rise of Artificial Intelligence: A Look into the Future

## Introduction
Artificial Intelligence (AI) is a rapidly evolving field that encompasses the development of intelligent machines capable of performing tasks that typically require human intelligence. The importance of AI cannot be understated, as it has found applications in various industries such as healthcare, finance, and retail, revolutionizing the way businesses operate. The key trends and players in the AI field continue to shape the future of technology and innovation.

## Latest Trends in Artificial Intelligence
Advancements in machine learning algorithms have significantly contributed to the progress of AI, enabling machines to learn from data and improve their performance over time. The increased adoption of AI in sectors like healthcare, finance, and retail has led to more efficient processes and improved decision-making. However, ethical considerations in AI devel

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: A Look into the Future

## Introduction
Artificial Intelligence (AI) is a rapidly evolving field that encompasses the development of intelligent machines capable of performing tasks that typically require human intelligence. The importance of AI cannot be understated, as it has found applications in various industries such as healthcare, finance, and retail, revolutionizing the way businesses operate. The key trends and players in the AI field continue to shape the future of technology and innovation.

## Latest Trends in Artificial Intelligence
Advancements in machine learning algorithms have significantly contributed to the progress of AI, enabling machines to learn from data and improve their performance over time. The increased adoption of AI in sectors like healthcare, finance, and retail has led to more efficient processes and improved decision-making. However, ethical considerations in AI development, such as bias in algorithms and data privacy, are important factors that need to be addressed. The impact of AI on the job market and workforce automation is also a topic of debate, with concerns about job displacement and the need for upskilling the workforce.

## Key Players in Artificial Intelligence
Companies like Google, Amazon, and IBM are leading the way in AI research and development, investing heavily in cutting-edge technologies and solutions. Startups are also disrupting the AI landscape with innovative ideas and approaches, driving competition and pushing boundaries in the industry. Academic institutions play a crucial role in contributing to AI innovations through research and education, fostering collaboration and knowledge-sharing within the academic community.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI technology, such as advancements in natural language processing and computer vision, are pushing the boundaries of what AI can achieve. Regulatory updates and policy changes affecting the AI industry, including guidelines on data privacy and algorithm transparency, are shaping the future of AI development. Success stories of AI implementation in businesses highlight the tangible benefits of integrating AI solutions into operations, from cost savings to improved customer experiences.

## Target Audience Analysis
Professionals in the tech industry seeking updates on AI trends, business owners looking to integrate AI solutions into their operations, students and researchers interested in AI advancements, and the general audience curious about the impact of AI on society will find valuable insights in this blog post. By catering to a diverse audience with varying levels of interest and expertise in AI technology, this article aims to provide a comprehensive overview of the latest trends, key players, and noteworthy news in the field of Artificial Intelligence.

In conclusion, Artificial Intelligence is poised to reshape the future of technology and innovation, with key trends, players, and news shaping the landscape of AI development. By staying informed and engaged with the latest advancements in AI, individuals and businesses can leverage the power of AI to drive growth and innovation in their respective fields. As AI continues to evolve, ethical considerations and regulatory frameworks will play a crucial role in ensuring the responsible and sustainable development of AI technologies. Stay tuned for more updates and insights on Artificial Intelligence trends, top AI companies, AI news updates, machine learning advancements, and ethical AI development.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "AI Agents Examples for Production purpose"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI Agents Examples for Production purpose.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer:
Content Plan Document:

Title: AI Agents Examples for Production: Latest Trends and Key Players

Introduction:
- Brief overview of AI agents and their importance in production processes.
- Mention of the rapid advancements in AI technology and its impact on various industries.
- Purpose of the article to provide examples of AI agents for production purposes.

Key Points:
1. Latest Trends in AI Agents for Production:
- Rise of autonomous AI agents in manufacturing and supply chain management.

I now can give a great answer

Final Answer:
# AI Agents Examples for Production: Latest Trends and Key Players

In the ever-evolving technological world, AI agents have become essential tools for optimizing production processes in various industries. The incorporation of artificial intelligence in production management has led to significant advancements in efficiency, quality control, and resource allocation. This article delves into the latest trends in AI agents for production purposes, shining a light on key players in AI technology development and providing real-world examples of successful implementation.

## Latest Trends in AI Agents for Production

One of the predominant trends in AI agents for production is the emergence of autonomous systems in manufacturing and supply chain management. These intelligent agents can analyze vast amounts of data in real-time, facilitating proactive decision-making and enhancing operational efficiency. Moreover, the utilization of AI agents in

In [15]:
Markdown(result)

# AI Agents Examples for Production: Latest Trends and Key Players

In the ever-evolving technological world, AI agents have become essential tools for optimizing production processes in various industries. The incorporation of artificial intelligence in production management has led to significant advancements in efficiency, quality control, and resource allocation. This article delves into the latest trends in AI agents for production purposes, shining a light on key players in AI technology development and providing real-world examples of successful implementation.

## Latest Trends in AI Agents for Production

One of the predominant trends in AI agents for production is the emergence of autonomous systems in manufacturing and supply chain management. These intelligent agents can analyze vast amounts of data in real-time, facilitating proactive decision-making and enhancing operational efficiency. Moreover, the utilization of AI agents in predictive maintenance and quality control has transformed how companies manage assets and ensure product quality. By harnessing AI technology, production facilities can predict maintenance requirements and detect potential defects before they escalate, ultimately boosting overall efficiency and minimizing downtime.

Another significant trend in the realm of AI agents for production is their integration into virtual environments to improve efficiency. Virtual production environments driven by AI agents provide a dynamic platform for simulating production scenarios, optimizing workflows, and pinpointing areas for enhancement. By leveraging AI in virtual settings, companies can experiment with various production strategies, forecast outcomes, and make informed decisions to maximize productivity. The seamless integration of AI agents in virtual production environments underscores the transformative potential of artificial intelligence in reshaping conventional production practices.

## Key Players in AI Agents Development

Numerous tech giants have played a pivotal role in advancing AI agents for production purposes, showcasing the diverse applications of AI technology in enhancing operational efficiency. Google's AI agents are recognized for optimizing production schedules and resource allocation through sophisticated algorithms that streamline processes and reduce waste. IBM's Watson AI platform is another prominent player, offering predictive analytics and real-time decision-making capabilities for production management. By harnessing AI, IBM empowers companies to make data-driven decisions, streamline workflows, and enhance overall productivity.

Amazon's Alexa AI has also left a mark in production management, particularly in voice-controlled systems and inventory management. Alexa's intuitive voice commands and seamless integration with production systems have revolutionized how companies interact with technology, providing a user-friendly interface for managing inventory, tracking shipments, and overseeing production processes. The innovative applications of Alexa AI in production settings highlight the potential of artificial intelligence to enhance operational efficiency and simplify daily tasks.

## Noteworthy News on AI Agents for Production

Recent case studies have demonstrated the successful integration of AI agents in production facilities, showcasing the tangible benefits of AI technology in optimizing efficiency and reducing operational costs. Companies across various industries have reported significant enhancements in productivity, quality control, and resource management following the implementation of AI agents. These success stories underscore the transformative influence of AI technology on traditional production practices, paving the way for a more efficient and sustainable future.

Looking ahead, future projections indicate that AI agents will continue to play a crucial role in reshaping the production landscape, driving innovation, and creating new growth opportunities. As advancements in AI technology progress, companies must embrace the potential of AI agents to remain competitive in today's fast-paced business environment. By leveraging AI in production settings, companies can unlock new efficiencies, enhance decision-making processes, and foster sustainable growth in the digital era.

In conclusion, AI agents are poised to revolutionize production management, offering a multitude of opportunities to enhance operational efficiency, quality control, and resource optimization. As companies increasingly adopt AI technology to drive innovation and streamline production processes, the significance of AI agents in reshaping the production landscape will only continue to grow. By staying informed about the latest trends and key players in AI technology development, companies can harness the power of AI agents to stay ahead of the curve and propel sustainable growth in the digital age.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).